In [2]:
import MySQLdb as mdb
import numpy as np
import pandas as pd
import re


class DataReader(object):
    '''A unified data reader interface to interact with the In-Part database.'''
    
    def __init__(self):
        """Connect to db and create a cursor for subsequent queries. """
        self.con = mdb.connect(host='localhost', user='root', passwd="123", db="capstone", local_infile=1) 
        self.cur = self.con.cursor() 
        
        # Other auxiliary instance variables
        self.user_num = None
        self.user_ids = None
        self.user_keywords = None
        
        self.tech_num = None
        self.tech_ids = None
        self.tech_keywords = None
        
        self.all_keywords = None
        self.keyword_mapping_dict = None
        
        self.tech_mapping_dict = None

    def get_user_num(self):
        """Return the number of unique users.
        
        Returns
        -------
        long
        """
        if self.user_num is not None:
            return self.user_num
        query = "SELECT DISTINCT id FROM users"
        self.cur.execute(query)
        self.user_num =  self.cur.rowcount
        return long(self.user_num)

    def get_user_ids(self):
        """Return a list of all user ids in the database. 
        
        Returns
        -------
        list
            A list of long like [1L, 2L, 3L].
        """
        if self.user_ids is not None:
            return self.user_ids
        query = "SELECT DISTINCT id FROM users"
        self.cur.execute(query)
        rows = self.cur.fetchall()
        self.user_ids = [row[0] for row in rows]
        return self.user_ids


    def get_tech_num(self):
        """Return the number of unique technologies in the database.
        
        Returns
        -------
        long        
        """
        if self.tech_num is not None:
            return self.tech_num
        query = "SELECT DISTINCT id FROM technologies"
        self.cur.execute(query)
        self.technology_num = self.cur.rowcount
        return self.technology_num

    def get_tech_ids(self):
        """Return a list of all technologies id in the database.
        
        Returns
        -------
        list
            A list of long.
        """
        if self.tech_ids is not None:
            return self.tech_ids
        query = "SELECT DISTINCT id FROM technologies"
        self.cur.execute(query)
        rows = self.cur.fetchall()
        self.tech_ids = [row[0] for row in rows]
        return self.tech_ids

    def get_tech_mapping_dict(self):
        '''Return a mapping dict which maps technology id to its index in the lis of all tech ids.
        
        Returns
        -------
        dict
        '''
        if self.tech_mapping_dict is not None:
            return self.tech_mapping_dict
        all_tech_ids = self.get_tech_ids()
        self.tech_mapping_dict = {value: index for index, value in enumerate(all_tech_ids)}
        return self.tech_mapping_dict
    
    
    def get_all_keywords(self):
        """Return a list of all keywords.
        
        Returns
        -------
        list 
            A list of long.
        """
        if self.all_keywords is not None:
            return self.all_keywords
        query = "SELECT DISTINCT id FROM keywords"
        self.cur.execute(query)
        rows = self.cur.fetchall()
        self.all_keywords = [row[0] for row in rows]
        return self.all_keywords
        
    def get_tech_keywords(self):
        """Return a dictionary whose key is technology id and whose value is a list of associated keywords.
        
        Returns
        -------
        dict 
        """
        if self.tech_keywords is not None:
            return self.tech_keywords
        query = "SELECT * FROM technology_keywords"
        self.cur.execute(query)
        self.tech_keywords = {}
        rows = self.cur.fetchall()
        for row in rows:
            if row[1] not in self.tech_keywords:
                self.tech_keywords[row[1]] = []
                self.tech_keywords[row[1]].append(row[0])
            else:
                self.tech_keywords[row[1]].append(row[0])
        return self.tech_keywords
    
    def get_user_keywords(self):
        """Return a dictionary whose key is user id and whose value is a list of associated keywords.
        
        Returns
        -------
        dict 
        """
        if self.user_keywords is not None:
            return self.user_keywords
        query = "SELECT * FROM user_keywords"
        self.cur.execute(query)
        self.user_keywords = {}
        rows = self.cur.fetchall()
        for row in rows:
            if row[1] not in self.user_keywords:
                self.user_keywords[row[1]] = []
                self.user_keywords[row[1]].append(row[0])
            else:
                self.user_keywords[row[1]].append(row[0])
        return self.user_keywords
        
        
    def get_keyword_mapping_dict(self):
        '''Return a mapping dict which maps keyword id to its index in the technology/user keyword vector.
        
        Note
        ----
        A technology id has many associated keywords. 
        A technology keyword vector of a technology id is a vector where the value of at that location of a keyword is 1 
        if this technology id has that keyword.
        
        A user keyword vector has the same principle.
        '''
        if self.keyword_mapping_dict is not None:
            return self.keyword_mapping_dict
        all_keywords = self.get_all_keywords()
        self.keyword_mapping_dict = {value: index for index, value in enumerate(all_keywords)}
        return self.keyword_mapping_dict
    
    
    def get_tech_keyword_vector(self, tech_id):
        '''Return the technology keyword vector given a technology id.
        
        Params
        ------
        tech_id: long
        
        Returns
        -------
        list
            A list of int.
        '''
        tech_keyword_vector = np.zeros(len(self.get_all_keywords()), dtype=np.int8)
        tech_keywords = self.get_tech_keywords().get(tech_id, None)
        if tech_keywords is None:       # No keywords associated with this technology
            return tech_keyword_vector
        keyword_mapping_dict = self.get_keyword_mapping_dict()
        indices = [keyword_mapping_dict[keyword] for keyword in tech_keywords]
        tech_keyword_vector[indices] = 1
        return tech_keyword_vector
    
    def get_user_keyword_vector(self, user_id):
        '''Return the user keyword vector given a user id.
        
        Params
        ------
        user_id: str
        
        Returns
        -------
        list
            A list of int.
        '''
        user_keyword_vector = np.zeros(len(self.get_all_keywords()), dtype=np.int8)
        user_keywords = self.get_user_keywords().get(user_id, None)
        if user_keywords is None:       # No keywords associated with this user
            return user_keyword_vector
        keyword_mapping_dict = self.get_keyword_mapping_dict()
        indices = [keyword_mapping_dict[keyword] for keyword in user_keywords]
        user_keyword_vector[indices] = 1
        return user_keyword_vector    
    
    def get_score_data(self):
        """Returns the score table.
        
        Returns
        -------
        DataFrame
            A table which contains columns user_id, technology_id, total_score.
        """
        self.scoreData = pd.read_sql("SELECT user_id, technology_id, total_score FROM score", con = self.con)        
        return self.scoreData
    
    def get_interacted_tech_ids(self, user_id):
        """Return a list of technology ids with which this user has interacted.
        
        Params
        ------
        user_id: str
        
        Returns
        -------
        list
        """         
        query = "SELECT technology_id FROM score WHERE user_id = '%s'" % user_id
        self.cur.execute(query)              
        rows = self.cur.fetchall()
        return [row[0] for row in rows]
     
    def get_contacted_tech_ids (self, user_id):
        """Return a list of contacted technology ids given a user id.
        
        Returns
        -------
        list
            A list of long.
        """
        query ="SELECT technology_id FROM contacts WHERE user_id = '%s'" % user_id
        self.cur.execute(query) 
        rows = self.cur.fetchall() 
        return [row[0] for row in rows]
    
    
    def get_clicked_tech_ids (self, user_id):
        """Return a list of clicked technology ids in all emails containing recommended tech ids given a user id.
        
        Returns
        -------
        list
            A list of long.
        """
        query ="SELECT clicked_technology_id FROM email_clicks WHERE user_id = '%s'" % user_id
        self.cur.execute(query) 
        rows = self.cur.fetchall() 
        return [row[0] for row in rows]
    
    def get_university_tech_ids(self, university_id):
        """Return a list of technology ids associated with this given university id.
        
        Returns
        -------
        list
            A list of long.
        """
        query = "SELECT id FROM technologies WHERE university_profile_id = '%s'" % university_id   
        self.cur.execute(query)
        rows = self.cur.fetchall()
        return [row[0] for row in rows]   
    
    def get_orphan_tech_ids(self):
        """Find orphan tech ids in user_activtivies table.
        
        Returns
        -------
        list
            tech_ids which appear in the user_activities table, but not in the technologies table
        """   
        activities = self.get_activities_table()
        viewed_tech_ids = []
        for index, row in activities.iterrows():
            start = row[1].find("Article_id") # Finding start from "Article_id"
            end = row[1].find("content")
            tech_id = int(re.search(r'\d+', row[1][start:end]).group(0))
            viewed_tech_ids.append(tech_id)     
        tech_ids = self.get_tech_ids() 
        orphan_tech_ids = list(set(tech_ids) - set(viewed_tech_ids))
        return orphan_tech_ids

    def get_contacts_table(self):
        """Return the contact table.
        
        Returns
        -------
        DataFrame 
            A table which contains columns user_id, technology_id, number of contacts.
        """
        self.contacts = pd.read_sql( "SELECT user_id, technology_id , count(*) as c_count FROM contacts group by user_id, technology_id", con = self.con)    
        return self.contacts
    
    def get_clicks_table(self):
        """Return the click table.
        
        Returns
        -------
        DataFrame   
            A table which contains columns user_id, technology_id, number of clicks.
        """
        self.clicks =  pd.read_sql( "SELECT user_id, clicked_technology_id as technology_id, count(*) as e_count FROM email_clicks group by user_id, technology_id", con = self.con)
        return self.clicks

    def get_activities_table(self):
        """Return the activities table.
        
        Returns
        -------
        DataFrame   
            A table which contains columns user_id, details.
        """
        self.activities = pd.read_sql( "SELECT user_id, details FROM user_activities", con = self.con) 
        return self.activities   


In [15]:
dr = DataReader()

In [4]:
dr.get_user_num()

1261L

In [5]:
dr.get_user_ids()[:5]

['525fea18-db1c-451d-9902-469ad4718e13',
 '525ff28d-2d28-4cf2-a4d8-468bd4718e13',
 '5260234c-9878-4d49-9d26-46b2d4718e13',
 '5260f967-7d74-4f6e-9deb-4b7fd4718e13',
 '526131a8-dbc8-401b-989a-44afd4718e13']

In [291]:
len(dr.get_user_keywords())

964

In [292]:
# dr.get_user_keywords()

In [293]:
dr.get_user_keywords().get('526132a9-4280-4cc4-a2d8-42acd4718e13')

In [294]:
dr.get_user_keyword_vector('526132a9-4280-4cc4-a2d8-42acd4718e13').sum()

0

In [16]:
dr.get_tech_num(), dr.get_tech_ids()[:5]

(1338L, [2L, 3L, 4L, 5L, 6L])

In [296]:
len(dr.get_tech_mapping_dict()), max([item[1] for item in dr.get_tech_mapping_dict().items()])

(849, 848)

In [297]:
#dr.get_tech_keywords()

In [298]:
dr.get_tech_keyword_vector(2L)

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [299]:
dr.get_all_keywords()[:5]

[1L, 2L, 3L, 4L, 5L]

In [300]:
len( dr.get_all_keywords())

7429

In [301]:
dr.get_clicked_tech_ids('57d97d4a-23b8-4148-a0a5-004a0a2ae3a6')

[35L]

In [302]:
dr.get_contacted_tech_ids('532a4687-92a4-4b5b-9d92-4f05d4718e13')

[60L, 40L, 23L, 64L, 41L, 47L, 51L]

In [303]:
#dr.get_orphan_tech_ids()

In [304]:
len(dr.get_orphan_tech_ids())

111

In [305]:
len(set(dr.get_tech_ids()) & set(dr.get_orphan_tech_ids()))

111

In [306]:
len(dr.get_tech_ids())

849

In [307]:
dr.get_score_data()['user_id'].unique().shape

(804,)

In [308]:
dr.get_score_data()['technology_id'].unique().shape

(738,)

In [69]:
cosine_similarity?

In [7]:
import pandas as pd
import numpy as np
import math
import itertools
import DataReader as dr
from sklearn.metrics.pairwise import cosine_similarity 
from scipy import sparse
from datetime import datetime

In [20]:
all_set = 0
item_set = 0
null_set = 0

In [28]:


class EnsembleRecommenderForTechnology(object):
    '''An ensemble recommender to recommend technologies for a tehchnology.
    
    Frist, calculate technology similarities between technology keyword vector.
    Second, calculate technology similarities between columns of utility matrix. Each technology vector is represented by scores given by users.
    Finally, do a weighted average of two technology matrix. The weights are [0.5, 0.5].
    '''
    def __init__(self):
        self.dr = DataReader()
        
        self.score = self.dr.get_score_data()
        self.tech_keyword_matrix = self.get_tech_keyword_matrix() 
        
        self.tech_keyword_sim_matrix = None
        self.item_based_sim_matrix = None
        self.ensemble_sim_matrix = None
        
        # Auxiliary variable
        self.tech_id_set_for_item_based = None
    
    def build_tech_keyword_sim_matrix(self):
        if self.tech_keyword_sim_matrix is not None:
            return self.tech_keyword_sim_matrix
        
        print "Statr building tech keyword sim matrix..."
        self.tech_keyword_matrix = sparse.csr_matrix(self.tech_keyword_matrix)
        self.tech_keyword_sim_matrix = pd.DataFrame(cosine_similarity(self.tech_keyword_matrix),
                                                    index=self.dr.get_tech_ids(),
                                                    columns=self.dr.get_tech_ids())
        print "Done."
        print '\n'
        
    def build_item_based_cl_sim_matrix(self):
        if self.item_based_sim_matrix is not None:
            return self.item_based_sim_matrix
        
        print "Statr building item_based Collaborative Filtering sim matrix..."
        scoreData = self.score
        score_df = scoreData.pivot(index = 'user_id', columns = 'technology_id', values = 'total_score') # Reshape score table 
        score_df = score_df.fillna(0) # fill NaN data with 0

        # Calculate Technology based similarity
        score_df_t = score_df.T
        score_spare_t = sparse.csr_matrix(score_df_t) 

        similarities_tech = cosine_similarity(score_spare_t)
        similarities_tech_df = pd.DataFrame(similarities_tech, columns=score_df_t.index, index=score_df_t.index)
        self.item_based_sim_matrix = similarities_tech_df
        self.tech_id_set_for_item_based = set(self.item_based_sim_matrix.index)

        print "Done."
        print '\n'    

    def build_ensemble_model(self):
        if self.ensemble_sim_matrix is not None:
            return self.ensemble_sim_matrix
        
        print "Statr building Ensemble sim matrix..."
        ensemble_sim_matrix = (self.tech_keyword_sim_matrix + self.item_based_sim_matrix) / 2
        ensemble_sim_matrix = ensemble_sim_matrix.loc[self.dr.get_tech_ids(), self.dr.get_tech_ids()]        

        # Replace null values of Item based sim matrix with similarity values in Technology keyword sim matrix
        all_tech_ids = set(self.dr.get_tech_ids())
        tech_ids_null_set = all_tech_ids - self.tech_id_set_for_item_based        
        for tech_id in tech_ids_null_set:
            ensemble_sim_matrix.loc[tech_id] = self.tech_keyword_sim_matrix.loc[tech_id]
            ensemble_sim_matrix.loc[:, tech_id] = self.tech_keyword_sim_matrix.loc[:, tech_id]
        
        self.ensemble_sim_matrix = ensemble_sim_matrix
        print "Done."
        print '\n'  
        
    def ensemble_recommend(self, tid, k):
        '''Return top k recommendations for a user id.
        
        Params
        ------
        tid: int
            technology id
        k: int
        
        Returns
        -------
        list
        '''
        tech_sim = zip(self.ensemble_sim_matrix.loc[tid], self.dr.get_tech_ids())
        tech_sim = sorted(tech_sim, key=lambda x: x[0], reverse=True)[1:k+1]
        return [tech_id for sim, tech_id in tech_sim]
    
    def save_recommendations_to_database(self, k):
        '''Create a table and Write top k recommendations for technology to the database.
        
        Params
        ------
        k: int
            The number of recommendations for each technology.
        '''
        print "Start writing recommendations to the database..."
        tech_ids = self.dr.get_tech_ids()
        recommendations = pd.DataFrame([self.ensemble_recommend(tid, k) for tid in tech_ids], 
                                       index=tech_ids)
        
        recommendations.to_csv('ensemble_recommendations_for_techs.txt', sep='\t', header=False)
        
        top_k_str = ""
        for i in range(1, k + 1):
            top_k_str += "top_" + str(i) + " int, "
        self.dr.cur.execute("DROP TABLE IF EXISTS RecommendationResultForTechs;")
        self.dr.cur.execute('''CREATE TABLE RecommendationResultForTechs (
                                   technology_id int,
                                   %s);''' % top_k_str[:-2])
        self.dr.cur.execute('''LOAD DATA LOCAL INFILE 'ensemble_recommendations_for_techs.txt' 
                                   INTO TABLE RecommendationResultForTechs 
                                   FIELDS TERMINATED BY '\t' 
                                   LINES TERMINATED BY '\n';''')
        self.dr.con.commit()
        print "Done"
    
    def get_tech_keyword_matrix(self):
        '''Return a technology keyword matrix whose row is a technology keyword vector.
        
        Returns
        -------
        A list of list
        '''
        all_tech_ids = self.dr.get_tech_ids()
        tech_keyword_matrix = []     # Dimension: num_techs * num_keywords
        for tech_id in all_tech_ids:
            tech_keyword_matrix.append(self.dr.get_tech_keyword_vector(tech_id))
        return tech_keyword_matrix

if __name__ == "__main__":
    recommender = EnsembleRecommenderForTechnology()
    recommender.build_tech_keyword_sim_matrix()
    recommender.build_item_based_cl_sim_matrix()
    recommender.build_ensemble_model()
    recommender.save_recommendations_to_database(10)

Statr building tech keyword sim matrix...
Dim of item: (1338, 1338)
Done.


Statr building item_based Collaborative Filtering sim matrix...
item set set([2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 20

In [19]:
1164 + 178

1342

In [25]:
len(item_set)

1164

In [26]:
len(null_set)

178

In [27]:
len(recommender.dr.get_tech_ids())

1338

In [275]:
class EnsembleRecommender(object):
    '''An ensemble recommender integrated from two content-based models and one item-based collaborative flitering.'''
    
    def __init__(self):
        self.dr = DataReader()
        
        self.score = self.dr.get_score_data()
        self.cb_matrix = None
        self.interacted_cb_matrix = None
        self.cl_matrix = None
        self.ensemble_matrix = None
        self.tech_keyword_matrix = self.get_tech_keyword_matrix()
        
    def build_content_based(self):
        '''Build a content-based model which calculates similarity between user keyword vector and tech keyword vector.'''
        if self.cb_matrix is not None:
            return self.cb_matrix
        
        print 'Start training content_based model...'
        start = datetime.now()
        
        all_user_ids = self.dr.get_user_ids()
        user_keyword_matrix = []     # Dimension: num_users * num_keywords
        for uid in all_user_ids:
            user_keyword_matrix.append(self.dr.get_user_keyword_vector(uid))
        
        # Convert matrix to sparse matrix to speed up similarity calculation
        user_keyword_matrix = sparse.csr_matrix(user_keyword_matrix)
        self.tech_keyword_matrix = sparse.csr_matrix(self.tech_keyword_matrix)
        self.cb_matrix = cosine_similarity(user_keyword_matrix, self.tech_keyword_matrix)  # Dim: num_users * num_techs
        print 'Done'
        print 'Time elapsed:', datetime.now() - start, '\n'
        
    def build_interacted_content_based(self):
        '''Build a intereacted cb model which calculates similarity between user preference vector and tech keyword vector.
        
        The user preference vector is a weighted average of tech keyword vectors with which users have interacted.
        '''
        if self.interacted_cb_matrix is not None:
            return self.interacted_cb_matrix
        
        self.tech_keyword_matrix = self.tech_keyword_matrix.toarray()
        
        print 'Start training interacted content_based model...'
        start = datetime.now()
        all_user_ids = self.dr.get_user_ids()
        tech_mapping_dict = self.dr.get_tech_mapping_dict()
        user_keyword_matrix = []
        for uid in all_user_ids:
            interacted_tech_ids = self.dr.get_interacted_tech_ids(uid)
            num = np.zeros(len(self.dr.get_all_keywords()), dtype=np.float32)
            deno = 0
            for tech_id in interacted_tech_ids:
                d= self.score[(self.score['user_id'] == uid) & (self.score['technology_id'] == tech_id)]['total_score']
    
                weight = d.values[0]
                num += weight * self.tech_keyword_matrix[tech_mapping_dict[tech_id], :]
                deno += weight
            if deno == 0:    # This user hasn't interacted with any technology
                user_keyword_matrix.append(self.dr.get_user_keyword_vector(uid))
            else:
                user_keyword_matrix.append(num / deno)

        # Convert matrix to sparse matrix to speed up similarity calculation
        user_keyword_matrix = sparse.csr_matrix(user_keyword_matrix)
        self.tech_keyword_matrix = sparse.csr_matrix(self.tech_keyword_matrix)
        self.interacted_cb_matrix = cosine_similarity(user_keyword_matrix, self.tech_keyword_matrix)  # Dim: num_users * num_techs
        print 'Done'
        print 'Time elapsed:', datetime.now() - start, '\n'
            
    def build_collaborative_filtering(self):
        '''Built a collaborative filtering model using score table.'''
        if self.cl_matrix is not None:
            return self.cl_matrix
        
        scoreData = self.score
        score_df = scoreData.pivot(index = 'user_id', columns = 'technology_id', values = 'total_score') # Reshape score table 
        score_df = score_df.fillna(0) # fill NaN data with 0

        # Calculate Technology based similarity
        score_df_t = score_df.T
        score_spare_t = sparse.csr_matrix(score_df_t) 

        similarities_tech = cosine_similarity(score_spare_t)
        similarities_tech_df = pd.DataFrame(similarities_tech, columns = score_df_t.index, index = score_df_t.index)

        # Create a list of user_ids and tech_ids which are available in score table
        user_ids = scoreData['user_id'].unique()
        tech_ids = scoreData['technology_id'].unique()

        # Calculate mean of all ratings, mean rating given by each user and mean rating given to each technology
        all_mean = np.mean(scoreData['total_score'])
        avg_user = {}
        for user in user_ids:
            avg_user[user] = np.mean(scoreData[scoreData['user_id'] == user]['total_score']) - all_mean

        avg_tech = {}
        for tech in tech_ids:
            avg_tech[tech] = np.mean(scoreData[scoreData['technology_id'] == tech]['total_score'])-all_mean
        
        
        print 'Start training collaborative filtering model...'
        start = datetime.now()
        count = 0
        base_line = {}
        prediction = {} # A dictionary storing predictions whose key is a tuple (user_id, tech_id)
        
        print 'Total num of calculation:', len(user_ids) * len(tech_ids)
        for i in itertools.product(user_ids,tech_ids):
            prediction[i] = score_df.ix[i[0],i[1]]
            base_line[i] = avg_user[i[0]] + avg_tech[i[1]] + all_mean
            if prediction[i] == 0:
                numerator = sum((score_df.ix[i[0]] - base_line[i])*similarities_tech_df.ix[i[1]])
                denominator = sum(similarities_tech_df.ix[i[1]])-1
                if denominator == 0:
                    prediction[i] = 0
                else:
                    prediction[i] = base_line[i] + numerator/float(denominator)
                    count += 1
                    if count % 100000 == 0:
                        print 'Num of calcuation finished:', count,
                        print '\tTime elapsed:', datetime.now() - start

        idx = pd.MultiIndex.from_tuples(prediction.keys())
        item_based_cf = pd.DataFrame(list(prediction.values()),index = idx,columns = ['Technology_id']).unstack(fill_value = 0)['Technology_id']
        item_based_cf = item_based_cf.loc[self.dr.get_user_ids(), self.dr.get_tech_ids()].fillna(0)
        self.cl_matrix = item_based_cf.values
        print 'Done'
        print 'Time elapsed:', datetime.now() - start, '\n'
        
    
    def build_ensemble_model(self, weights):
        '''Build an ensemble model.
        
        Params
        ------
        weights: list
            Ensemble weights of item_based cf, content_based, interacted_content_based.
        '''
        if self.ensemble_matrix is not None:
            return self.ensemble_matrix
        index = self.dr.get_user_ids()
        cols = self.dr.get_tech_ids()
        norm_item_based_cf = self.normalize_df(pd.DataFrame(self.cl_matrix, index=index, columns=cols))
        norm_cb = self.normalize_df(pd.DataFrame(self.cb_matrix, index=index, columns=cols))
        norm_interacted_cb = self.normalize_df(pd.DataFrame(self.interacted_cb_matrix, index=index, columns=cols))
        
        weighted_ensemble = norm_item_based_cf * weights[0] + norm_cb * weights[1] + norm_interacted_cb * weights[2]
        self.ensemble_matrix = weighted_ensemble
    
    def ensemble_recommend(self, uid, k):
        '''Return top k recommendations for a user id.
        
        Returns
        -------
        tuple
            A tuple (a list of top k recommendations, a list of tech ids clicked in emails)
        '''
        user_preds = zip(self.ensemble_matrix.loc[uid], self.dr.get_tech_ids())
        user_preds = sorted(user_preds, key=lambda x: x[0], reverse=True)
        
        contacted_ids = self.dr.get_contacted_tech_ids(uid)
        top_k = [item_id for score, item_id in user_preds if item_id not in contacted_ids][:k]
        
        clicked_tech_ids = self.dr.get_clicked_tech_ids(uid)
        email_clicked = [tech_id for tech_id in top_k if long(tech_id) in clicked_tech_ids]
        return top_k, email_clicked

    def save_recommendations_to_database(self, k):
        '''Create a table and Write top k recommendations for each user to the database.
        
        Params
        ------
        k: int
            The number of recommendations for each user.
        '''
        uids = self.dr.get_user_ids()
        result = [self.ensemble_recommend(uid, k) for uid in uids]
        recommendations = [item[0] for item in result]
        email_clicked = [item[1] for item in result]
        output = pd.concat([pd.DataFrame({'user_id': uids}), pd.DataFrame(recommendations), pd.DataFrame({'email_clicked': email_clicked})], axis=1)
        output.to_csv('ensemble_recommendations_for_users.txt', sep='\t', index=False, header=False)
        
        top_k_str = ""
        for i in range(1, k + 1):
            top_k_str += "top_" + str(i) + " int, "
        self.dr.cur.execute("DROP TABLE IF EXISTS RecommendationResultForUsers;")
        self.dr.cur.execute('''CREATE TABLE RecommendationResultForUsers (
                                   user_id VARCHAR(60),
                                   %s 
                                   email_clicked VARCHAR(220));''' % top_k_str)
        self.dr.cur.execute('''LOAD DATA LOCAL INFILE 'ensemble_recommendations_for_users.txt' 
                                   INTO TABLE RecommendationResultForUsers 
                                   FIELDS TERMINATED BY '\t' 
                                   LINES TERMINATED BY '\n';''')
        self.dr.con.commit()

    
    def get_tech_keyword_matrix(self):
        '''Return a technology keyword matrix whose row is a technology keyword vector.
        
        Returns
        -------
        A list of list
        '''
        all_tech_ids = self.dr.get_tech_ids()
        tech_keyword_matrix = []     # Dimension: num_techs * num_keywords
        for tech_id in all_tech_ids:
            tech_keyword_matrix.append(self.dr.get_tech_keyword_vector(tech_id))
        return tech_keyword_matrix
    
    def normalize_df(self, score_matrix):
        '''Normalize the range of each user's score to [0,1].
        
        Params
        ------
        score_matrix: DataFrame
            Row is user, Column is technology, Value is the score. Recommendations are made by ranking scores for a user.
        
        Returns
        -------
        DataFrame
        '''
        norm_df = score_matrix.copy()
        for idx, row in norm_df.iterrows():
            min_val = min(row)
            max_val = max(row)
            interval = max_val - min_val
            if interval == 0:
                continue
            norm_df.ix[idx] = [(r - min_val) / interval for r in row]
        return norm_df
    
if __name__ == "__main__":
    ensemble_weights = [0.65, 0.67, 0.52]
    recommender = EnsembleRecommender()
    recommender.build_content_based()
    recommender.build_interacted_content_based()
    recommender.build_collaborative_filtering()
    recommender.build_ensemble_model(ensemble_weights)
    recommender.save_recommendations_to_database(10)
   

Start training content_based model...
Done
Time elapsed: 0:00:00.156338 

Start training interacted content_based model...
Done
Time elapsed: 0:00:20.021226 

Start training collaborative filtering model...
Total num of calculation: 593352
Num of calcuation finished: 100000 	Time elapsed: 0:01:14.511157
Num of calcuation finished: 200000 	Time elapsed: 0:02:27.499231
Num of calcuation finished: 300000 	Time elapsed: 0:03:36.634235
Num of calcuation finished: 400000 	Time elapsed: 0:04:50.123190
Num of calcuation finished: 500000 	Time elapsed: 0:06:05.845653
Done
Time elapsed: 0:07:08.014944 



/Users/Chaofan/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:181: Warning: Unknown table 'capstone.recommendationresultforusers'


In [207]:
from DataReaderplus import *
cols = weighted_ensemble.columns
dr = DataReader()

def ensemble_recommend(uid, k):
    ''' Returns top k recommendations for a user id '''
    user_preds = zip(weighted_ensemble.loc[uid], cols)
    user_preds = sorted(user_preds, key=lambda x: x[0], reverse=True)
    contact_ids = dr.get_contacted_tech_ids(uid)
    top_k = [item_id for score, item_id in user_preds if item_id not in contact_ids][:k]
    return top_k

if __name__ == '__main__':
    time = datetime.now()
    uids = user_pool
    recommendations = [ensemble_recommend(uid, 5) for uid in uids]
    df = pd.DataFrame(recommendations)
    email_clicked = []
    for i, uid in enumerate(uids):
        email_clicked.append([int(tech_id)  for tech_id in recommendations[i] if long(tech_id) in dr.get_clicked_tech_ids(uid)])
    output = pd.concat([pd.DataFrame({'user_id': uids}), df, pd.DataFrame({'email_clicked': email_clicked})], axis=1)
    output.to_csv('ensemble_recommendations.csv', index=False)
    print 'Time elapsed: ', datetime.now() - time

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [257]:
e = EnsembleRecommender()

In [258]:
e.build_content_based()
e.build_interacted_content_based()

Start training content_based model...
Done
Time elapsed: 0:00:00.191848 

Start training interacted content_based model...
Done
Time elapsed: 0:00:15.971555 



In [259]:
e.build_collaborative_filtering()

Start training collaborative filtering model...
Total num of calculation: 593352
Num of calcuation finished: 100000 	Time elapsed: 0:01:21.140684
Num of calcuation finished: 200000 	Time elapsed: 0:02:37.001720
Num of calcuation finished: 300000 	Time elapsed: 0:03:56.585435
Num of calcuation finished: 400000 	Time elapsed: 0:05:12.438050
Num of calcuation finished: 500000 	Time elapsed: 0:06:33.723254
Done
Time elapsed: 0:07:45.710770 



In [261]:
e.build_ensemble_model([0.65, 0.67, 0.52])

In [263]:
e.ensemble_matrix.shape

(977, 849)

In [270]:
len('525fea18-db1c-451d-9902-469ad4718e13')

36

In [264]:
e.ensemble_matrix.index

Index([u'525fea18-db1c-451d-9902-469ad4718e13',
       u'525ff28d-2d28-4cf2-a4d8-468bd4718e13',
       u'5260234c-9878-4d49-9d26-46b2d4718e13',
       u'5260f967-7d74-4f6e-9deb-4b7fd4718e13',
       u'526131a8-dbc8-401b-989a-44afd4718e13',
       u'526132a9-4280-4cc4-a2d8-42acd4718e13',
       u'526640ab-ebd8-4593-9749-4002d4718e13',
       u'52698ff9-0470-47c7-b027-d9c9d4718e13',
       u'5283a1d7-dae0-4d34-b38f-493cd4718e13',
       u'5283aeaa-d9e4-46ec-a318-3a1ed4718e13',
       ...
       u'5876357e-50a8-4584-bdbd-00860a2ac68a',
       u'5877557e-bbac-4e3f-944a-00810a2a6136',
       u'58775f13-5944-46d8-9ec7-00bd0a2a6136',
       u'587788f1-a39c-473f-8294-008a0a2a6136',
       u'5877e33e-21f8-472f-be30-00b70a2ac68a',
       u'58788ee8-bd40-4c20-992f-008a0a2a6136',
       u'5878982f-92f8-4803-8c4a-008a0a2a6136',
       u'5878c1c5-fe60-4018-9a89-00810a2a6136',
       u'5878f2d1-a1a8-4155-b583-00320a2a9d99',
       u'5879fa50-eecc-4812-8892-002f0a2a2cca'],
      dtype='object', length

In [266]:
e.ensemble_recommend('57ea95e3-5d44-4330-81df-00500a2abcf8', 10)

([504L, 516L, 253L, 503L, 783L, 510L, 168L, 807L, 365L, 499L],
 [504L, 516L, 253L, 503L, 510L, 168L])

In [244]:
e.cl_matrix.shape

(977, 849)

In [176]:
dr.get_score_data()[()'user_id'=='528f575a-c6a8-4fdb-9bd7-4662d4718e13') & 'technology_id' == 17L]['total_score']

SyntaxError: invalid syntax (<ipython-input-176-09420479894e>, line 1)

In [179]:
s = dr.get_score_data()
s[(s['user_id']=='528f575a-c6a8-4fdb-9bd7-4662d4718e13') & (s['technology_id'] == 17L)]['total_score'].values[0]

5.0

In [125]:
e = EnsembleRecommender()
e.build_content_based()

In [115]:
dr.get_tech_keyword_vector(3l)

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [76]:
sparse.csr_matrix?

In [108]:
cosine_similarity?

In [153]:
np.array(e.tech_keyword_matrix.toarray(), dtype=np.float32)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [156]:
e.tech_keyword_matrix.toarray()[1, :].sum()

16

In [157]:
np.zeros(10, dtype=np.int8) + np.ones(10, dtype=np.float32)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

In [159]:
np.zeros(10).dtype

dtype('float64')